# AG CROP PLANNING TOOL

The Ag Decision Engine accepts inputs from a user (Crop Type, Location and Timeframe) and develops a customized crop planning and protection plan for farmland owners or operatators across North Carolina. The decision engine offers a basic interface for user input and leverages ouputs from a crop performance prediction model and a RAG-enhanced LLM for recomendation building.

## CONTENT
1.0 - Initial Setup\
2.0 - Crop Prediction Model\
3.0 - Decision Logic Model\
4.0 - Recomendation Builder\
5.0 - User Interface\
6.0 - Launch Tool

## 1.0 INITIAL SETUP

**OVERVIEW:** Function to run user inputs through appropriate crop prediction models, transform results and pass data to Decision Logic Model.

**IMPORTS** _(General Purpose)_

In [112]:
import path # supports file paths
import os # supports use of environment variables
import pandas as pd # supports use of dataframes
import numpy as np # supports mathmatical functionality
import json

# Supports progress monitoring features
import time 
from tqdm import tqdm

# Removes unnecessary warnings
import warnings
warnings.filterwarnings('ignore')

### 1.1 General Environment Setup

**DEPENDENCIES**
* For Natural Lanugage Processing
    * Local LLM:  Ollama _([download]('https://ollama.com/download/windows'))_ running 'phi3:mini' model _([documentation]('https://ollama.com/library/phi3'))_
    * Hosted LLM: OpenAI _([documentation]('https://platform.openai.com/docs/overview'))_ 
* For Document Loading, Embedding and Retrieval (RAG Functionality)
    * LangChain _([documentation]('https://python.langchain.com/v0.2/docs/introduction/')) loads and splits documents_
    * Unstructured _([documentation]('https://docs.unstructured.io/welcome')) pre-processes pdf documents_
    * OpenAI _([documentation]('https://platform.openai.com/docs/guides/embeddings/')) converts documents into embeddings_
    * ChromaDB _([documentation]('https://docs.trychroma.com/getting-started')) stores embeddings_

**INSTRUCTIONS**
1. Load or call selected LLM
2. _(Option A, default)_ For running LLM locally:  
    *  Download the [Ollama servce]((https://ollama.com/download)), if needed.
    *  Start the Ollama service by running the following command:`ollama serve`
    *  Allow Ollama service to run in the background while running code
    *  Pull the latest update the Ollama [phi3-mini](https://ollama.com/library/phi3) model by running the following command:`ollama pull phi3:mini`
2. _(Option B)_ For using hosted LLM (e.g., OpenAI, Claude, etc.):
    *  Update the code in Section 1.1 below. (Code available for running OpenAI LLM; for other models, make similar code edits)
3. Initalize LLM 


### 1.2 LLM Selection and Setup

In [6]:
# Assumes use of local LLM (if using hosted LLM use code block below)
import ollama
from langchain.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.embeddings import OllamaEmbeddings

# For Ollama (local LLM)
llm = Ollama(model="phi3:mini")

In [7]:
# UNCOMMENT CODE below to use OpenAI hosted LLM (OpenAI) 

# import openai # for hosted LLM option
# from langchain import OpenAI
# from langchain.embeddings import OpenAIEmbeddings
# from dotenv import load_dotenv, find_dotenv
# _ = load_dotenv(find_dotenv()) # reads local .env file
# openai.api_key = os.environ['OPENAI_API_KEY']

# llm = OpenAI()

### 1.3 RAG System Setup

**Helper Function for HTML Handling**

In [8]:
# Helper function to support html files docs with different encodings
class CustomHTMLLoader(UnstructuredFileLoader):
    def __init__(self, file_path: str):
        super().__init__(file_path)

    def _get_elements(self):
        try:
            with open(self.file_path, 'r', encoding='utf-8') as f:
                content = f.read()
        except UnicodeDecodeError:
            try:
                with open(self.file_path, 'r', encoding='latin-1') as f:
                    content = f.read()
            except UnicodeDecodeError:
                with open(self.file_path, 'r', encoding='cp1252') as f:
                    content = f.read()
        
        soup = BeautifulSoup(content, 'html.parser')
        text = soup.get_text(separator='\n', strip=True)
        return [text]

**Helper Function for PDF Handling**

In [9]:
# Helper function to support loading text files with different encodings
class CustomTextLoader(UnstructuredFileLoader):
    def __init__(self, file_path: str):
        super().__init__(file_path)

    def _get_elements(self):
        try:
            with open(self.file_path, 'r', encoding='utf-8') as f:
                text = f.read()
        except UnicodeDecodeError:
            try:
                with open(self.file_path, 'r', encoding='latin-1') as f:
                    text = f.read()
            except UnicodeDecodeError:
                with open(self.file_path, 'r', encoding='cp1252') as f:
                    text = f.read()
        return [text]

**Setup RAG Document Repository**

In [10]:
# Checks current directory path (helps user ensure correct documents_path set
current_dir = os.getcwd()
print("Current working directory:", current_dir)

Current working directory: C:\Users\Jamie\OneDrive\desktop\AI_Bootcamp\MOD_23_Project_3\AgProject3


In [11]:
# Loads documents from current working directory
documents_path = './rag_content' # EDIT PATH FOR NEW DIRECTORY AS NEEDED

**Document Loading**

In [12]:
# Sets up loaders for different file types
loaders = {
    "**/*.pdf": PyPDFLoader,
    "**/*.html": CustomHTMLLoader,
    "**/*.txt": CustomTextLoader
}
# Error handling - checks if the directory exists
if not os.path.exists(documents_path):
    print(f"Directory not found: {documents_path}")
    print("Contents of current directory:")
    print(os.listdir(os.getcwd()))
    raise FileNotFoundError(f"Directory {documents_path} does not exist")

print(f"Directory found: {documents_path}")

# Selects appropriate loader
def get_loader(file_path):
    for glob_pattern, loader_class in loaders.items():
        if file_path.endswith(glob_pattern.split("*")[-1]):
            return loader_class(file_path)
    return CustomTextLoader(file_path)  # Default to CustomTextLoader

# Loads documents
print("Loading documents...")
documents = []
errors = []

for root, _, files in os.walk(documents_path):
    for file in tqdm(files, desc="Processing files"):
        file_path = os.path.join(root, file)
        try:
            loader = get_loader(file_path)
            docs = loader.load()
            documents.extend(docs)
        except Exception as e:
            errors.append((file_path, str(e)))

print(f"Loaded {len(documents)} documents")
print(f"Encountered {len(errors)} errors")

if errors:
    print("\nErrors encountered:")
    for file_path, error in errors:
        print(f"{file_path}: {error}")

# Displays summary of loaded documents
file_types = {}
for doc in documents:
    file_type = os.path.splitext(doc.metadata.get('source', ''))[-1].lstrip('.')
    file_types[file_type] = file_types.get(file_type, 0) + 1

print("\nSummary of loaded documents:")
for file_type, count in file_types.items():
    print(f"{file_type}: {count}")

Directory found: ./rag_content
Loading documents...


Processing files: 100%|████████████████████████████| 6/6 [00:00<00:00,  7.17it/s]

Loaded 10 documents
Encountered 0 errors

Summary of loaded documents:
pdf: 7
html: 3


**Text Spliting and Content Chunking**

In [13]:
# Initializes the text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

# Initializes an empty list to store the split documents
start_time = time.time()
split_docs = []
total_chunks = 0

# Supports monitoring progress
for i, doc in enumerate(tqdm(documents, desc="Splitting documents")):
    
    # Prints count number of document being processed size of the document before processing
    print(f"\nProcessing document {i+1}/{len(documents)}")
    print(f"Document {i+1} size: {len(doc.page_content)} characters")
    
    # Performs the text splitting
    doc_start_time = time.time()
    split_doc = text_splitter.split_documents([doc])
    split_docs.extend(split_doc)
    
    # Calculates and prints statistics for the current document
    doc_time = time.time() - doc_start_time
    chunks_created = len(split_doc)
    total_chunks += chunks_created
    
    print(f"Document {i+1}/{len(documents)} processed:")
    print(f"  - Chunks created: {chunks_created}")
    print(f"  - Time taken: {doc_time:.2f} seconds")
    
    # Error handling - avoids division by zero
    if doc_time > 0:
        print(f"  - Processing speed: {len(doc.page_content) / doc_time:.2f} characters/second")
    else:
        print(f"  - Processing speed: N/A (processed too quickly to measure)")
    
    print(f"Total time elapsed: {time.time() - start_time:.2f} seconds")

# Records final statistics
total_time = time.time() - start_time
total_characters = sum(len(doc.page_content) for doc in documents)

print("\nText splitting complete!")
print(f"Total documents processed: {len(documents)}")
print(f"Total chunks created: {total_chunks}")
print(f"Total characters processed: {total_characters}")
print(f"Total time taken: {total_time:.2f} seconds")

# Avoid division by zero in overall statistics
if total_time > 0:
    print(f"Overall processing speed: {total_characters / total_time:.2f} characters/second")
else:
    print("Overall processing speed: N/A (processed too quickly to measure)")

# Now split_docs contains all the split documents
docs = split_docs

Splitting documents: 100%|█████████████████████| 10/10 [00:00<00:00, 2250.04it/s]


Processing document 1/10
Document 1 size: 5276 characters
Document 1/10 processed:
  - Chunks created: 7
  - Time taken: 0.00 seconds
  - Processing speed: 3668017.22 characters/second
Total time elapsed: 0.00 seconds

Processing document 2/10
Document 2 size: 6632 characters
Document 2/10 processed:
  - Chunks created: 9
  - Time taken: 0.00 seconds
  - Processing speed: N/A (processed too quickly to measure)
Total time elapsed: 0.00 seconds

Processing document 3/10
Document 3 size: 1079 characters
Document 3/10 processed:
  - Chunks created: 2
  - Time taken: 0.00 seconds
  - Processing speed: N/A (processed too quickly to measure)
Total time elapsed: 0.00 seconds

Processing document 4/10
Document 4 size: 18854 characters
Document 4/10 processed:
  - Chunks created: 24
  - Time taken: 0.00 seconds
  - Processing speed: 18681646.02 characters/second
Total time elapsed: 0.00 seconds

Processing document 5/10
Document 5 size: 2475 characters
Document 5/10 processed:
  - Chunks create

**Text Embedding**

In [14]:
# Generates embeddings for the document chunks
print("Generating embeddings and creating vector store...")
start_time = time.time()

embeddings = OllamaEmbeddings(model="phi3:mini")

# Creates a progress bar for tracking progress
pbar = tqdm(total=len(docs), desc="Processing documents")

def embed_function(texts):
    results = embeddings.embed_documents(texts)
    pbar.update(len(texts))
    return results

# Create the vector store
vector_store = Chroma.from_documents(
    documents=docs,
    embedding=embeddings
)

pbar.close()

end_time = time.time()
total_time = end_time - start_time

print(f"\nEmbedding generation and vector store creation completed.")
print(f"Total time taken: {total_time:.2f} seconds")
print(f"Average time per document: {total_time/len(docs):.2f} seconds")

Generating embeddings and creating vector store...


Processing documents:   0%|                               | 0/67 [02:46<?, ?it/s]


Embedding generation and vector store creation completed.
Total time taken: 166.53 seconds
Average time per document: 2.49 seconds


**Setup Retriver**

In [15]:
# Creates a retriever using the vector store
retriever = vector_store.as_retriever()

## 2.0 CROP PREDICTION MODEL
**OVERVIEW**: Function to run user inputs through appropriate crop prediction models, transform results and pass data to Decision Logic Model.

**DEPENDENCIES**
* Python
* SciKit Learn
* Prediction models _(see [Resources](./Resources) folder)_

**IMPORTS** _(Prediction Model)_ 

In [38]:
import pickle # For accessing modeling results
from sklearn.preprocessing import StandardScaler, MinMaxScaler # for transforming model inputs

# ML models used for crop predictions
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

### 2.1 Crop Prediction Function

In [39]:
# Sets location of trained ML models
folder = './Resources/'

In [40]:
# Function to call appropriate prediction model and return results
def crop_prediction(measure,factorarray):
    
    modelmap = {
    'BARLEY_$_ACRE' : ['BARLEY_$_ACREgbr_model.pkl','M', 213.2],
    'BARLEY_BU_ACRE' : ['BARLEY_BU_ACREdtr_model.pkl','M', 69.4],
    'CORN_$_ACRE' : ['CORN_$_ACREsvr_model.pkl','L', 512.2],
    'CORN_BU_ACRE' : ['CORN_BU_ACREgbr_model.pkl','M', 116.8],
    'COTTON_LB_ACRE' : ['COTTON_LB_ACREsvr_model.pkl','M', 829.6],
    'HAY_$_ACRE' : ['HAY_$_ACRErfr_model.pkl','M', 228.86],
    'HAY_T_ACRE' : ['HAY_T_ACREsvr_model.pkl','M', 2.28],
    'OATS_$_ACRE' : ['OATS_$_ACREsvr_model.pkl','M', 192.9],
    'OATS_BU_ACRE' : ['OATS_BU_ACREsvr_model.pkl','M', 67.625],
    'PEANUTS_$_ACRE' : ['PEANUTS_$_ACREsvr_model.pkl','L', 882.9],
    'PEANUTS_LB_ACRE' : ['PEANUTS_LB_ACRErfr_model.pkl','H', 3575.4],
    'PEPPERS, BELL_CWT_ACRE' : ['PEPPERS, BELL_CWT_ACREsvr_model.pkl','L', 209.6],
    'PEPPERS,BELL_$_ACRE' : ['PEPPERS,BELL_$_ACREsvr_model.pkl','H', 7712.7],
    'SOYBEANS_$_ACRE' : ['SOYBEANS_$_ACREgbr_model.pkl','M', 321.5],
    'SOYBEANS_BU_ACRE' : ['SOYBEANS_BU_ACRElr_model.pkl','H', 33.3],
    'SQUASH_$_ACRE' : ['SQUASH_$_ACREsvr_model.pkl','L', 3567.6],
    'SQUASH_CWT_ACRE' : ['SQUASH_CWT_ACREdtr_model.pkl','H', 110],
    'SWEET_$_ACRE' : ['SWEET_$_ACREsvr_model.pkl','L', 2954.9],
    'SWEET_CWT_ACRE' : ['SWEET_CWT_ACREdtr_model.pkl','L', 117.3],
    'TOBACCO_$_ACRE' : ['TOBACCO_$_ACREgbr_model.pkl','H', 3925.6],
    'TOBACCO_LB_ACRE' : ['TOBACCO_LB_ACREsvr_model.pkl','M', 2119.2],
    'WHEAT_$_ACRE' : ['WHEAT_$_ACREsvr_model.pkl','M', 267.3],
    'WHEAT_BU_ACRE' : ['WHEAT_BU_ACREgbr_model.pkl','H', 53]
    }


   #print('entering crop_prediction with',measure,'and',factorarray)

    if measure not in modelmap.keys():
        print('invalid measure',measure,'. Valid measures are\n',modelmap.keys())
        return None
    
    modelfile = folder + modelmap[measure][0]
    #print(modelfile)
    scalar_X_file = modelfile.replace("model","X")
    scalar_y_file = modelfile.replace("model","y")
    #print('about to open files',modelfile,scalar_X_file,scalar_y_file)

    with open(modelfile, 'rb') as file:
        #print('opened',modelfile)
        loaded_model = pickle.load(file)

    with open(scalar_X_file, 'rb') as file:
        #print('opened',scalar_X_file)
        loaded_X_scaler = pickle.load(file)

    with open(scalar_y_file, 'rb') as file:
        #print('opened',scalar_y_file)
        loaded_y_scaler = pickle.load(file)

    #print('loaded',modelfile,scalar_X_file,scalar_y_file)
    
    factorarray = np.array(factorarray).reshape(1, -1)
    #display(factorarray.shape)

    # When making predictions:
    X_scaled = loaded_X_scaler.transform(factorarray)  # Scale new input data
    y_pred_scaled = loaded_model.predict(X_scaled)
    y_pred = loaded_y_scaler.inverse_transform(y_pred_scaled.reshape(-1,1))  # Inverse transform predictions
    
    conf = modelmap[measure][1]
    avg20 = modelmap[measure][2]

    #Returns crop name, predicted $/acre value, 20-average $/acre value and L/M/H modeling confidence 
    return measure.split('_')[0], y_pred[0][0], avg20, conf

## 3.0 DECISION LOGIC MODEL

**OVERVIEW**: Function to take predicted $/acre peformance of selected crop(s) with respective 20-year average performance and determine if a crop should be planted, planted with caution or not planted. Includes LLM call to generate decision justification narrative.

**IMPORTS** _(Decision Logic)_

In [97]:
from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough

### 3.1 Decisioning and Labeling Function 

In [98]:
# Takes in dictionary of crops' predicted $/acre, 20-year average $/acre and H/M/L modeling confidence
# Returns dictionary with justification narrative added 

def make_crop_decision(prediction_dict):
    # Initialize the Ollama model
    llm = Ollama(model="phi3:mini")

    # Creates a prompt template to generate supporting narrative for decision and label
    decision_prompt_template = PromptTemplate(
        input_variables=["crop", "action", "prediction_percentage", "confidence"],
        template="Generate a brief narrative explaining why the crop {crop} is labeled as '{action}'. "
                 "The prediction is {prediction_percentage}% of the 20-year average, "
                 "and the confidence level is {confidence}. "
                 "Provide considerations for planting based on these factors."
    )

    # Create the RunnableSequence
    decision_chain = decision_prompt_template | llm

    crop_decisions = {}

    for crop, data in prediction_dict.items():
        prediction = data['prediction']
        avg20 = data['average_20_year']
        confidence = data['confidence']

        # Calculate the prediction as a percentage of the 20-year average
        prediction_percentage = (prediction / avg20) * 100

        # Assign initial label based on prediction percentage
        if prediction_percentage > 85:
            action = 'Plant'
        elif 60 <= prediction_percentage <= 80:
            action = 'Plant with caution'
            if confidence == 'L':
                action += ' (low confidence)'
        else:
            action = 'Do not plant. Consider alternatives'

        # Generate narrative using the LLM
        decision_narrative = decision_chain.invoke({
            "crop": crop,
            "action": action,
            "prediction_percentage": prediction_percentage,
            "confidence": confidence
        })

        crop_decisions[crop] = {
            'Action': action,
            'Considerations': decision_narrative.strip()
        }

    return crop_decisions

## 4.0 RECOMENDATION BUILDER
**OVERVIEW**: Accepts dataframe variable containing crop performance and associated justifications. Generates recommendation narrative for each crop using LLM. Supplements recommendation with additional considerations and mitagation information retrieve from RAG.


**IMPORTS** _(Recommendation Builder)

In [99]:
# for loading various document types
from langchain_community.document_loaders import PyPDFLoader, BSHTMLLoader, UnstructuredFileLoader, DirectoryLoader
from bs4 import BeautifulSoup
from langchain.chains import LLMChain
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import DirectoryLoader

# Libraries for prompting and parsing
from langchain.prompts import PromptTemplate
from langchain.output_parsers import RegexParser
from langchain.schema.runnable import RunnablePassthrough

# Libraries for Output Parser
import re
import nltk
from nltk.tokenize import sent_tokenize

# Download necessary NLTK data
#nltk.download('punkt')


#### 4.1 Recommendation Prompt Template

In [100]:
# Define the prompt template
results_prompt_template = PromptTemplate(
    input_variables=["context", "query", "crop_data"],
    template="""You are an agricultural specialist who advises farmers on how to optimize farm operations and mitigate against weather and climate disasters. 

Use the following context to inform your answer:

Context: {context}

Question: {query}

Crop Data: {crop_data}

Please provide a detailed response, creating a brief narrative for each crop and its respective action. Include considerations shown and supplement with financial risk mitigation strategies or crop resilience advice for each crop.

Answer: """
)

# Create an LLMChain
results_llm_chain = results_prompt_template | llm

### 4.2 Recommendation Generator

In [109]:
# Function to create risk mitagation strategies narrative to accompany decision and justification narrative (raw_results)
def generate_recommendations(crop_decisions):

    query = """Review the provided crop decisions dictionary and for each crop and respective action, create a brief narrative to describe the considerations shown. Supplement the considerations with any financial risk mitigation strategies or crop resilience advice for the respective crop."""
    
    # Retrieves relevant guidence and risk management strategies using the RAG retriever
    relevant_docs = retriever.get_relevant_documents(query)
    context = "\n".join([doc.page_content for doc in relevant_docs])
    
    # Run the chain
    result = results_llm_chain.invoke({
        "context": context,
        "query": query,
        "crop_data": str(crop_decisions)
    })
    
    return result

In [113]:
# Combines output output of Decisioning and Labeling function and pairs with Supplimental Information
def generate_final_recommendation(results_dict):
    query = """Review the provided crop data dictionary and for each crop and respective action, create a brief narrative to describe the considerations shown. Supplement the considerations with any financial risk mitigation strategies or crop resilience advice for the respective crop."""
    
    # Retrieve relevant context using the retriever
    relevant_docs = retriever.get_relevant_documents(query)
    context = "\n".join([doc.page_content for doc in relevant_docs])
    
    # Run the chain
    recommendation_narrative = results_llm_chain.invoke({
        "context": context,
        "query": query,
        "crop_data": str(results_dict)
    })
    
    return recommendation_narrative

## 5.0 USER INTERFACE
**OVERVIEW**: Creates Gradio user enterface to enable user to select a count, select crops to consider and input 4-digit planting year using keyboard.


**IMPORTS** _(User Interface)_

In [114]:
# Uses Gradio to build interface
import gradio as gr

### 5.1 Input Definitions

In [115]:
# Defines the inputs for counties, crops and seasons
counties = ["Alamance", "Alexander", "Alleghany", "Anson", "Ashe", "Avery", "Beaufort", "Bertie", "Bladen", "Brunswick",
            "Buncombe", "Burke", "Cabarrus", "Caldwell", "Camden", "Carteret", "Caswell", "Catawba", "Chatham",
            "Cherokee", "Chowan", "Clay", "Cleveland", "Columbus", "Craven", "Cumberland", "Currituck", "Dare",
            "Davidson", "Davie", "Duplin", "Durham", "Edgecombe", "Forsyth", "Franklin", "Gaston", "Gates", "Graham",
            "Granville", "Greene", "Guilford", "Halifax", "Harnett", "Haywood", "Henderson", "Hertford", "Hoke", "Hyde",
            "Iredell", "Jackson", "Johnston", "Jones", "Lee", "Lenoir", "Lincoln", "Macon", "Madison", "Martin",
            "McDowell", "Mecklenburg", "Mitchell", "Montgomery", "Moore", "Nash", "New Hanover", "Northampton",
            "Onslow", "Orange", "Pamlico", "Pasquotank", "Pender", "Perquimans", "Person", "Pitt", "Polk", "Randolph",
            "Richmond", "Robeson", "Rockingham", "Rowan", "Rutherford", "Sampson", "Scotland", "Stanly", "Stokes",
            "Surry", "Swain", "Transylvania", "Tyrrell", "Union", "Vance", "Wake", "Warren", "Washington", "Watauga",
            "Wayne", "Wilkes", "Wilson", "Yadkin", "Yancey"]

crops = ['Barley', 'Corn', 'Hay', 'Oats', 'Peanuts', 'Bell Peppers', 'Soybeans', 'Squash',
         'Sweet Potatoes', 'Tobacco', 'Wheat']

seasons = ['Spring', 'Summer', 'Fall']


### 5.2 Input Transformation

NOTE: for demonstration purposes, we simulate forecasted seasional avg temperatures (degrees F), seasional avg precipitation (inches), and weeks of D2, D3 and D4 drought conditions _(i.e., Avg Fall Temp , Avg Spring Temp, Avg Summer Temp, Avg Winter Temp,Avg Fall Precip, Avg Spring Precip, Avg Summer Precip, Avg Winter Precip, Weeks of Severe drought (D2), Weeks of Extreme Drought (D3), Weeks of Exceptional Drought)._

Actual forecast can be pulled in via API from Weather forecasting service.

**Weather Forecast Simulation Function**

In [116]:
# function simuate weather forecast data
def get_varied_factors():
   
    base_factors = [58.77, 59.8, 75.13, 42.6, 9.58, 11.03, 14.9, 8.83, 29, 15, 1]
    
    # Precipitation variation (first 4 values)
    precipitation = np.array(base_factors[:4])
    precipitation_variation = np.random.normal(0, 5, 4)  # Mean 0, std dev 5
    varied_precipitation = np.maximum(precipitation + precipitation_variation, 0)  # Ensure non-negative
    
    # Temperature variation (next 4 values)
    temperature = np.array(base_factors[4:8])
    temperature_variation = np.random.normal(0, 2, 4)  # Mean 0, std dev 2
    varied_temperature = temperature + temperature_variation
    
    # Weeks of drought variation (last 3 values)
    drought_weeks = np.array(base_factors[8:])
    drought_variation = np.random.randint(-2, 3, 3)  # Random integer between -2 and 2
    varied_drought = np.maximum(drought_weeks + drought_variation, 0)  # Ensure non-negative
    
    # Combines all varied factors
    varied_factors = np.concatenate([varied_precipitation, varied_temperature, varied_drought])

    # returns factors array
    return varied_factors.tolist()


**User Interface Function**

In [117]:
#Function to run the entire process (input ->prediction -> decision - > strategies -> recommendation narrative)
def user_interface(image, county, crop_list, selected_seasons, year):
    # Retrieves simulated weather forecast data
    factors = get_varied_factors()

    # Converts user input into crop_prediction function input
    crop_input = {
        'Barley': 'BARLEY_$_ACRE',
        'Corn': 'CORN_$_ACRE',
        'Hay': 'HAY_$_ACRE',
        'Oats': 'OATS_$_ACRE',
        'Peanuts': 'PEANUTS_$_ACRE',
        'Bell Peppers': 'PEPPERS,BELL_$_ACRE',
        'Soybeans': 'SOYBEANS_$_ACRE',
        'Squash': 'SQUASH_$_ACRE',
        'Sweet Potatoes': 'SWEET_$_ACRE',
        'Tobacco': 'TOBACCO_$_ACRE',
        'Wheat': 'WHEAT_$_ACRE'
    }
   
    # Creates dictionary to hold the prediction values for each crop
    prediction_dict = {}
   
    for crop in crop_list:
        # Function to convert crop name into CROPNAME_$_ACRE
        model_input = crop_input[crop]

        # Runs prediction model for given crop
        cropname, prediction, avg20, conf = crop_prediction(model_input, factors)

        # Appends results to prediction dictionary
        prediction_dict[crop] = {
            'prediction': float(prediction),  # Ensure this is a float
            'average_20_year': float(avg20),  # Ensure this is a float
            'confidence': conf
        }
    
    # Generates planting decisions with justification based on crop predictions
    crop_decisions = make_crop_decision(prediction_dict)

    # Expands decisions into full recommendations with risk mitigation strategies
    recommendations = generate_recommendations(crop_decisions)
    
    # Combine predicted $/acre, decisions, and recommendations
    results_dict = {
        'predictions': prediction_dict,
        'decisions': crop_decisions,
        'recommendations': recommendations
    }

    final_recommendation = generate_final_recommendation(results_dict)
      
    # Convert the final recommendation to a JSON-serializable format
    json_output = json.dumps({
        'county': county,
        'selected_seasons': selected_seasons,
        'year': year,
        'final_recommendation': final_recommendation
    }, default=str)
    
    return json_output

**Recomendation Generation Function**

# 6.0 Launch Tool

**Launches Gradio U/I**

In [118]:
# Define output to Gradio interface
outputs = gr.JSON(label="Crop Predictions and Recommendations")

# Launch the Gradio interface
gr.Interface(
    fn=user_interface,
    inputs=[
        gr.Image(value="Images/ui_image.png", label="Farm Image"),  
        gr.Dropdown(choices=counties, label="Select County"),
        gr.CheckboxGroup(choices=crops, label="Crops to Consider"),
        gr.CheckboxGroup(choices=seasons, label="Planting Season(s)", value=seasons),
        gr.Number(label="Planting Year (YYYY)", value=2025, minimum=2025, maximum=2035)
    ],
    outputs=outputs,
    title="Crop Planning and Protection Plan Generator"
).launch(share=True)

Running on local URL:  http://127.0.0.1:7870
Running on public URL: https://4b126e80b918f25189.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
